In [1]:
!pip install pydub

In [2]:
!pip install opencv-python
!pip install mediapipe
!pip install protobuf
!pip install autogui

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import random
import numpy as np

cap = cv2.VideoCapture(0)
hand_detector = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils
screen_width, screen_height = pyautogui.size()

SCREEN_WIDTH = int(cap.get(3))
SCREEN_HEIGHT = int(cap.get(4))

# Define a list to store finger tracking points for the index finger
index_finger_tracking = []

class GameObject:
    def __init__(self, x, y, shape, color):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = color
        self.speed = 10

    def chagecolor(self, newcolor):
        self.color = newcolor
        
    def move_down(self):
        self.y += self.speed

    def draw(self, frame):
        if self.shape == 'circle':
            cv2.circle(frame, (self.x, self.y), 20, self.color, -1)
        elif self.shape == 'square':
            cv2.rectangle(frame, (self.x - 20, self.y - 20), (self.x + 20, self.y + 20), self.color, -1)
        elif self.shape == 'triangle':
            triangle_points = [(self.x, self.y - 20), (self.x - 20, self.y + 20), (self.x + 20, self.y + 20)]
            cv2.fillPoly(frame, [np.array(triangle_points)], self.color)

# random game object
def generate_random_object():
    x = random.randint(20, SCREEN_WIDTH - 20)
    y = 0  # Start from the top of the screen
    shapes = ['circle', 'square', 'triangle']
    colors = [(0, 255, 0), (0, 0, 255), (0, 255, 255)]  # Green, Red, Yellow
    shape = random.choice(shapes)
    color = random.choice(colors)
    return GameObject(x, y, shape, color)

game_objects = []

index_tip = None
thumb_tip = None

score = 0

game_over = False

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks

    if not game_over:
        if hands:
            for hand in hands:
                drawing_utils.draw_landmarks(frame, hand, mp.solutions.hands.HAND_CONNECTIONS,
                                            landmark_drawing_spec=drawing_utils.DrawingSpec(color=(0, 0, 255),
                                                                                            thickness=2,
                                                                                            circle_radius=4),
                                            connection_drawing_spec=drawing_utils.DrawingSpec(color=(0, 0, 255),
                                                                                              thickness=2))
                landmarks = hand.landmark
                for id, landmark in enumerate(landmarks):
                    x = int(landmark.x * frame_width)
                    y = int(landmark.y * frame_height)

                    if id == 8:  # Index finger tip
                        index_tip = (x, y)
                        cv2.circle(img=frame, center=(x, y), radius=20, color=(225, 0, 0), thickness=-1)  # Enlarge the index finger and change color to blue
                        # Append the index finger tip to the tracking list
                        index_finger_tracking.append((x, y))
                    elif id == 4:  # Thumb tip
                        thumb_tip = (x, y)

                    # Limit the tracking list to the last N points (adjust N as needed)
                    N = 100
                    index_finger_tracking = index_finger_tracking[-N:]

                if index_tip and thumb_tip:
                    # Calculate the distance between the index finger tip and thumb tip
                    distance = ((index_tip[0] - thumb_tip[0]) ** 2 + (index_tip[1] - thumb_tip[1]) ** 2) ** 0.5

                    if distance < 40:
                        # Check for collision with game objects and update the score
                        for obj in game_objects:
                            if obj.x - 20 <= index_tip[0] <= obj.x + 20 and obj.y - 20 <= index_tip[1] <= obj.y + 20:
                                if obj.color == (0, 255, 0):  # Green circle adds 5 to score
                                    obj.color = (0, 255, 255)
                                    score += 5
                                elif obj.color == (0, 0, 255):  # Red square subtracts 3 from score
                                    obj.color = (0, 255, 255)
                                    score -= 3
                                elif obj.color == (0, 255, 255):  # Yellow triangle subtracts 2 from score
                                    obj.color = (0, 255, 255)
                                    score -= 2
                                # game_objects.remove(obj)

        # Draw tracking lines for the index finger's left-to-right movement
        if len(index_finger_tracking) > 1:
            for i in range(1, len(index_finger_tracking)):
                cv2.line(frame, index_finger_tracking[i - 1], index_finger_tracking[i], (0, 255, 0), 3)

        # Move and draw each game object
        for obj in game_objects:
            obj.move_down()
            obj.draw(frame)

            # Check if the object is out of the screen, then remove it and generate a new one
            if obj.y > SCREEN_HEIGHT:
                game_objects.remove(obj)

        # Generate new game objects if there are fewer than 5 on the screen
        while len(game_objects) < 3:  # Adjust the number of objects as needed
            game_objects.append(generate_random_object())

        # Display the player's score on the screen
        cv2.putText(frame, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Check if the game is over (score >= 20)
        if score >= 20:
            game_over = True
            cv2.putText(frame, "Game Over", (SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 255), 2)

    # Display the frame with game objects and score
    cv2.imshow('Tap The Shape Game:', frame)

    if game_over:
        # Wait for a key press, and break the loop when any key is pressed
        key = cv2.waitKey(0)
        if key != -1:
            break
    else:
        # Exit the loop if 'q' is pressed and the game is not over
        if cv2.waitKey(1) & 0xFF == ord('q') and not game_over:
            break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
